In [15]:
# 1. data
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

bs = 64

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dl = DataLoader(train_data, batch_size=bs)
test_dl = DataLoader(test_data, batch_size=bs)

for x, y in test_dl:
    print(f"shape of x [n, c, h, w]: {x.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break

shape of x [n, c, h, w]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [21]:
# 2. model selection
from torch import nn

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
# 3. training loop

def train(dl, model, l, optimizer):
    size = len(dl.dataset)
    model.train()

    for batch, (x,y) in enumerate(dl):
        x, y = x.to(device), y.to(device)

        # 1. evaluate empirical risk
        y_hat = model(x)
        loss = l(y_hat, y)

        # 2. step via backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")
        
def test(dl, model, l):
    size = len(dl.dataset)
    n = len(dl)
    model.eval()
    loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dl:
            x, y = x.to(device), y.to(device)
            
            y_hat = model(x)
            loss += l(y_hat, y).item()
            correct += (y_hat.argmax(1) == y).type(torch.float).sum().item()

            loss /= n
            correct /= size

In [25]:
# 4. training run
epochs = 5
l = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, l, optimizer)
    test(test_dl, model, l)

print("Done!")

Epoch 1
-------------------------------
loss:1.935572 [   64/60000]
loss:1.909261 [ 6464/60000]
loss:1.795659 [12864/60000]
loss:1.844228 [19264/60000]
loss:1.730565 [25664/60000]
loss:1.669249 [32064/60000]
loss:1.696086 [38464/60000]
loss:1.591831 [44864/60000]
loss:1.630408 [51264/60000]
loss:1.519489 [57664/60000]
Epoch 2
-------------------------------
loss:1.599690 [   64/60000]
loss:1.566420 [ 6464/60000]
loss:1.420597 [12864/60000]
loss:1.499237 [19264/60000]
loss:1.374233 [25664/60000]
loss:1.358594 [32064/60000]
loss:1.370794 [38464/60000]
loss:1.291904 [44864/60000]
loss:1.339661 [51264/60000]
loss:1.231419 [57664/60000]
Epoch 3
-------------------------------
loss:1.335901 [   64/60000]
loss:1.315019 [ 6464/60000]
loss:1.154890 [12864/60000]
loss:1.266905 [19264/60000]
loss:1.139239 [25664/60000]
loss:1.154360 [32064/60000]
loss:1.169653 [38464/60000]
loss:1.105049 [44864/60000]
loss:1.158310 [51264/60000]
loss:1.064325 [57664/60000]
Epoch 4
-------------------------------


In [31]:
# 5. inference
torch.save(model.state_dict(), "model.pth")
print("Saved weights to model.pth")

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    y_hat = model(x)
    actual, expected = classes[y_hat[0].argmax(0)], classes[y]
    print(f'actual: "{actual}", expected: "{expected}"')

Saved weights to model.pth
actual: "Ankle boot", expected: "Ankle boot"
